# Kernals

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score, roc_auc_score, make_scorer
from sklearn.model_selection import StratifiedKFold
import pandas as pd


from scripts.kernals import *
from scripts.nested_CV import nested_cv

In [2]:
# data
file_path = "../data/laser.mat"
mat = loadmat(file_path)

X = mat["X"]
y = mat["Y"].reshape(200)

print(X.shape, y.shape)

(200, 60) (200,)


In [3]:
# configs
seed = 42  # for repeatability
np.random.seed(42)

print(np.random.randn(2))

[ 0.49671415 -0.1382643 ]


In [4]:
# normalizing the data
X = (X - X.mean(axis=1, keepdims=True)) / X.std(axis=1, keepdims=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=seed
)

## 1. SVC with Polynomical Kernal

In [5]:
clf_poly = KernelEstimator(k_poly, 3, 100)
clf_poly.fit(X_train, y_train)
y_preds_rbf = clf_poly.predict(X_test)

In [18]:
clf_poly.fit(X, y)
y_preds_poly = clf_poly.predict(X)
print(y_preds_poly)
print(classification_report(y, y_preds_poly))

[ 1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1.  1.
 -1. -1.  1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1.
 -1. -1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1.
 -1.  1. -1. -1. -1.  1. -1. -1. -1. -1.  1.  1.  1. -1. -1. -1. -1.  1.
  1. -1. -1. -1.  1. -1. -1. -1. -1. -1.  1.  1. -1. -1. -1. -1. -1. -1.
 -1.  1. -1.  1. -1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1.
  1. -1. -1. -1. -1.  1. -1. -1.  1. -1. -1. -1.  1. -1.  1. -1. -1. -1.
 -1. -1.  1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1. -1. -1. -1. -1.
  1. -1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1. -1. -1. -1. -1. -1.  1.
  1. -1. -1. -1. -1. -1. -1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1.]
              precision    recall  f1-score   support

          -1       0.40      0.65      0.50       100
           1       0.10      0.04      0.06       100

    accuracy           

In [7]:
odd_laser_faulty = clf_poly.predict([X[35]])
odd_laser_non_faulty = clf_poly.predict([X[162]])
print(odd_laser_faulty, odd_laser_non_faulty)

[-1.] [1.]


## 2. SVC with RBF

In [8]:
clf_rbf = KernelEstimator(k_rbf, 2, 10)
clf_rbf.fit(X_train, y_train)
y_preds_rbf = clf_rbf.predict(X_test)

In [9]:
y_preds_rbf = clf_rbf.predict(X_test)
print(classification_report(y_test, y_preds_rbf))

              precision    recall  f1-score   support

          -1       0.64      0.82      0.72        17
           1       0.83      0.65      0.73        23

    accuracy                           0.72        40
   macro avg       0.73      0.74      0.72        40
weighted avg       0.75      0.72      0.73        40



In [10]:
odd_laser_faulty = clf_rbf.predict([X[35]])
odd_laser_non_faulty = clf_rbf.predict([X[162]])
print(odd_laser_faulty, odd_laser_non_faulty)

[-1.] [-1.]


## 3. SVC with DTW

In [11]:
clf_dtw = KernelEstimator(k2_hyp, 2, 10)
clf_dtw.fit(X_train, y_train)
y_preds_dtw = clf_dtw.predict(X_test)

In [12]:
y_preds_dtw = y_preds_dtw.astype(int)
print(classification_report(y_test, y_preds_dtw))

              precision    recall  f1-score   support

          -1       0.82      0.82      0.82        17
           1       0.87      0.87      0.87        23

    accuracy                           0.85        40
   macro avg       0.85      0.85      0.85        40
weighted avg       0.85      0.85      0.85        40



In [13]:
odd_laser_faulty = clf_dtw.predict([X[35]])
odd_laser_non_faulty = clf_dtw.predict([X[162]])
print(odd_laser_faulty, odd_laser_non_faulty)

[-1.] [-1.]
